# Loading NeuroArch Database with Medulla 7 Column Dataset

This tutorial provides code to load NeuroArch database with Medulla 7 Column Dataset. Requirement before running the notebook:
- Installed [NeuroArch](https://github.com/fruitflybrain/neuroarch), [OrientDB Community Version](https://www.orientdb.org/download), and [pyorient](https://github.com/fruitflybrain/pyorient). The [NeuroNLP Docker image](https://hub.docker.com/r/fruitflybrain/neuronlp) and [FlyBrainLab Docker image](https://hub.docker.com/r/fruitflybrain/fbl) all have a copy of the software requirement ready.
- Download the [Neuprint database dump for the Medulla 7 Column dataset](https://github.com/connectome-neuprint/neuPrint/raw/master/fib25_neo4j_inputs.zip).
- Have 1GB free disk space (for Neuprint dump and NeuroArch database).

A backup of the database created by this notebook can be downloaded [here](https://drive.google.com/file/d/1YTkoSwHeh0bIu8O-2Ek98rY0udoCb3MA/view?usp=sharing). To restore it in OrientDB, run
```
/path/to/orientdb/bin/console.sh "create database plocal:../databases/medulla admin admin; restore database /path/to/medulla7column_2b6d75f1c_na_v1.0_backup.zip"
```

In [ ]:
import glob
import os
import subprocess
import csv
from collections import Counter

import numpy as np
import pandas as pd
from tqdm import tqdm
import h5py

import neuroarch.models as models
import neuroarch.na as na

## Extract Neuron and Synapse Attributes

In [ ]:
def process(chunk):
    status = np.nonzero(np.array([i == 'Traced' for i in chunk['status:string'].values]))[0]
    used = chunk.iloc[status]
    neurons = []

    for i, row in used.iterrows():
        li = [row['bodyId:long'], row['pre:int'], row['post:int'], row['status:string'],\
              row['statusLabel:string'], int(row['cropped:boolean']) if not np.isnan(row['cropped:boolean']) else row['cropped:boolean'], row['instance:string'], \
              row['type:string']]
        neurons.append(li)
    return neurons

In [ ]:
chunksize = 100000

with open('neurons.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['bodyID','pre','post','status','statusLabel','cropped','instance','type'])
    df = pd.read_csv('fib25/Neuprint_Neurons_fib25.csv')
    neurons = process(df)
    writer.writerows(neurons)

In [ ]:
neurons = pd.read_csv('neurons.csv')
used = []
swc_dir = 'swc'
for i, row in neurons.iterrows():
    if os.path.exists('{}/{}.swc'.format(swc_dir, row['bodyID'])):
        if isinstance(row['instance'], str):
            if row['instance'] in ['glia', 'cell body']:
                continue
            used.append(i)
        elif isinstance(row['type'], str):
            used.append(i)



In [ ]:
traced_neuron_id = neurons.iloc[used]['bodyID'].to_numpy()
        
chunksize = 1000000
pre_syn = np.empty((int(1e7),3), np.int64)
post_syn = np.empty((int(1e7),3), np.int64)

pre_count = 0
post_count = 0
count = 0
for chunk in pd.read_csv('fib25/Neuprint_SynapseSet_to_Synapses_fib25.csv', chunksize=chunksize):
    ids = chunk[':START_ID']
    pre_site = np.array([[n, int(i.split('_')[0]), int(i.split('_')[1])] \
                         for n,i in enumerate(ids) if i.split('_')[2] == 'pre'])
    post_site = np.array([[n, int(i.split('_')[0]), int(i.split('_')[1])] \
                          for n,i in enumerate(ids) if i.split('_')[2] == 'post'])
    pre_site_known = pre_site[np.logical_and(
                              np.isin(pre_site[:,1], traced_neuron_id),
                              np.isin(pre_site[:,2], traced_neuron_id)),0]
    post_site_known = post_site[np.logical_and(
                                np.isin(post_site[:,1], traced_neuron_id),
                                np.isin(post_site[:,2], traced_neuron_id)),0]
    retrieved_pre_site = chunk.iloc[pre_site_known]
    pre_site = np.array([[row[':END_ID(Syn-ID)'], int(row[':START_ID'].split('_')[0]), int(row[':START_ID'].split('_')[1])] \
                         for i, row in retrieved_pre_site.iterrows()])
    retrieved_post_site = chunk.iloc[post_site_known]
    post_site = np.array([[row[':END_ID(Syn-ID)'], int(row[':START_ID'].split('_')[0]), int(row[':START_ID'].split('_')[1])] \
                         for i, row in retrieved_post_site.iterrows()])
    if pre_site.size:
        pre_syn[pre_count:pre_count+pre_site.shape[0], :] = pre_site
        pre_count += pre_site.shape[0]
    if post_site.size:
        post_syn[post_count:post_count+post_site.shape[0], :] = post_site
        post_count += post_site.shape[0]
    count += chunksize
    print(count, pre_count, post_count)

pre_syn = pre_syn[:pre_count,:]
post_syn = post_syn[:post_count,:]

ind = np.argsort(pre_syn[:,0])
pre_syn_sorted = pre_syn[ind, :]
ind = np.argsort(post_syn[:,0])
post_syn_sorted = post_syn[ind, :]


In [ ]:
# extract synapse (pre-site) to synapse (post-site) connection
# use only the post synaptic site to get all the synapses because one presynaptic site can have multiple postsynaptic sites
post_syn_index = post_syn_sorted[:,0].copy()

df = pd.read_csv('fib25/Neuprint_Synapse_Connections_fib25.csv')
post_ids = df[':END_ID(Syn-ID)']
used = np.where(post_ids.isin(post_syn_index).to_numpy())[0]
connections = df.iloc[used].to_numpy()
ind = np.argsort(connections[:,1])
connections = connections[ind, :]


In [ ]:
# extract synapse details
# with h5py.File('syn_pre_post_sorted_by_synapse_id.h5', 'r') as f:
#     pre_syn_sorted = f['pre_syn_sorted'][:]
#     post_syn_sorted = f['post_syn_sorted'][:]
chunksize = 100000

pre_syn_index = list(set(pre_syn_sorted[:,0].copy()))
pre_syn_index.extend(list(post_syn_sorted[:,0].copy()))
syn_index = np.array(sorted(pre_syn_index))
del pre_syn_index#, pre_syn_sorted, post_syn_sorted

synapse_array = np.empty((len(syn_index), 6), np.int64)

synapse_count = 0
count = 0

for chunk in pd.read_csv('fib25/Neuprint_Synapses_fib25.csv', chunksize=chunksize):
    ids = chunk[':ID(Syn-ID)']
    
    start_id = ids.iloc[0]
    stop_id = ids.iloc[-1]
    pre_start = np.searchsorted(syn_index, start_id, side='left')
    pre_end = np.searchsorted(syn_index, stop_id, side='right')
    if pre_start >= len(syn_index):
        pre_index = []
    else:
        if pre_end >= len(syn_index):
            pre_index = syn_index[pre_start:pre_end] #same as syn_index[pre_start:]
        else:
            pre_index = syn_index[pre_start:pre_end]
    pre_used_synapse = chunk.loc[ids.isin(pre_index)]
    li = np.empty((pre_index.size, 6), np.int64)
    i = 0
    for _, row in pre_used_synapse.iterrows():
        location = eval(row['location:point{srid:9157}'].replace('x', "'x'").replace('y', "'y'").replace('z', "'z'"))
        li[i,:] = [row[':ID(Syn-ID)'], # synpase id
                     0 if row['type:string'] == 'pre' else 1, #synapse type
                     int(row['confidence:float']*1000000), #confidence
                     location['x'], location['y'], location['z']]
        i += 1
    synapse_array[synapse_count:synapse_count+pre_index.shape[0],:] = li
    synapse_count += pre_index.shape[0]
    count += chunksize
    print(count, len(pre_used_synapse))
synapse_array = synapse_array[:synapse_count,:]


In [ ]:
# reorder synapses

synapse_connections = connections
    
ids = synapse_array[:,0]
syn_id_dict = {j: i for i, j in enumerate(ids)}
# ids = pre_syn_sorted[:,0]
# pre_syn_id_dict = {j: i for i, j in enumerate(ids)} # map syn id to pre_syn_sorted
ids = post_syn_sorted[:,0]
post_syn_id_dict = {j: i for i, j in enumerate(ids)} # map syn id to post_syn_sorted

synapse_dict = {}
wrong_synapse = 0
for i, pair in tqdm(enumerate(synapse_connections)):
    pre_syn_id = pair[0]
    post_syn_id = pair[1]
    post_id = post_syn_id_dict[post_syn_id]
    post_info = synapse_array[syn_id_dict[post_syn_id]]
    post_neuron_id, pre_neuron_id = post_syn_sorted[post_id, 1:]

    #if len(np.where((pre_syn_sorted == (pre_syn_id, pre_neuron_id, post_neuron_id)).all(axis=1))[0]) != 1:
    #    print(pre_syn_id, post_syn_id)
    # pre_id = pre_syn_id_dict[pre_syn_id]
    pre_info = synapse_array[syn_id_dict[pre_syn_id]]

    if pre_neuron_id not in synapse_dict:
        synapse_dict[pre_neuron_id] = {}
    pre_dict = synapse_dict[pre_neuron_id]
    if post_neuron_id not in synapse_dict[pre_neuron_id]:
        pre_dict[post_neuron_id] =  {'pre_synapse_ids': [],
                                     'post_synapse_ids': [],
                                     'pre_confidence': [],
                                     'post_confidence': [],
                                     'pre_x': [],
                                     'pre_y': [],
                                     'pre_z': [],
                                     'post_x': [],
                                     'post_y': [],
                                     'post_z': [],
                                     }
    info_dict = pre_dict[post_neuron_id]
    info_dict['pre_synapse_ids'].append(pre_syn_id)
    info_dict['post_synapse_ids'].append(post_syn_id)
    info_dict['pre_confidence'].append(pre_info[2])
    info_dict['post_confidence'].append(post_info[2])
    info_dict['pre_x'].append(pre_info[3])
    info_dict['pre_y'].append(pre_info[4])
    info_dict['pre_z'].append(pre_info[5])
    info_dict['post_x'].append(post_info[3])
    info_dict['post_y'].append(post_info[4])
    info_dict['post_z'].append(post_info[5])

with open('synapses.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['pre_id','post_id','N','pre_confidence','post_confidence',\
                     'pre_x','pre_y','pre_z','post_x','post_y','post_z'])
    for pre, k in tqdm(synapse_dict.items()):
        for post, v in k.items():
            writer.writerow([pre, post, len(v['pre_x']), str(v['pre_confidence']), \
                             str(v['post_confidence']), str(v['pre_x']), str(v['pre_y']), str(v['pre_z']), \
                             str(v['post_x']), str(v['post_y']), str(v['post_z'])])

## Load Data to NeuroArch

In [ ]:
medulla = na.NeuroArch('localhost', 'medulla', mode = 'o')

In [ ]:
species = medulla.add_species('Drosophila melanogaster', stage = 'adult',
                                sex = 'female',
                                synonyms = ['fruit fly', 'common fruit fly', 'vinegar fly'])

In [ ]:
version = '2b6d75f1c'
datasource = medulla.add_DataSource('Medulla7column', version = version,
                                      url = 'https://www.janelia.org/project-team/flyem/research/previous-connectomes-analyzed/seven-column-connectome-fib-sem',
                                      description = 'data obtained from https://github.com/connectome-neuprint/neuPrint/blob/922a107df827a2fedd671438595603c4d15eafa7/fib25_neo4j_inputs.zip; neuron skeleton from https://github.com/janelia-flyem/ConnectomeHackathon2015',
                                      species = species)
medulla.default_DataSource = datasource

In [ ]:
medulla.add_Neuropil('MED(L)', synonyms = ['left medulla'])

for i in range(1, 11):
    medulla.add_Subregion('MED-M{}(L)'.format(i),\
                          synonyms = ['left medulla M{} stratum'.format(i), 'left medulla stratum M{}'.format(i)],
                          neuropil = 'MED(L)')

In [ ]:
neuron_list = pd.read_csv('neurons.csv')
swc_dir = 'swc'
uname_dict = {}

for i, row in tqdm(neuron_list.iterrows()):
    if isinstance(row['instance'], str):
        if row['instance'] in ['glia', 'cell body']:
            continue
    elif isinstance(row['type'], str):
        pass
    else:
        continue

    bodyID = row['bodyID']
    cell_type = row['type']
    name = row['instance']
    
    if not os.path.exists('{}/{}.swc'.format(swc_dir, bodyID)):
        continue
    
    if not isinstance(name, str):
        if isinstance(cell_type, str):
            name = '{}-{}'.format(cell_type, bodyID)
        else:
            cell_type = 'unknown'
            name = 'unknown-{}'.format(bodyID)
    else:
        if not isinstance(cell_type, str):
            if name.split('-')[0] == 'tan':
                cell_type = 'tangential'
                if name not in uname_dict:
                    uname_dict[name] = 0
                uname_dict[name] += 1
                name = '{}-{}'.format(name, uname_dict[name])
            elif name.split('-')[0] == 'out':
                cell_type = 'output'
            elif name in ['Tm23/24', 'Tm23/24-F', 'Dm8-out', 'TmY16?']:
                cell_type = name
                if name not in uname_dict:
                    uname_dict[name] = 0
                uname_dict[name] += 1
                name = '{}-{}'.format(name, uname_dict[name])
            else:
                cell_type = name.split('-')[0]
        else:
            if name in ['Tm23/24', 'Tm23/24-F', 'Dm8-out', 'TmY16?']:
                if name not in uname_dict:
                    uname_dict[name] = 0
                uname_dict[name] += 1
                name = '{}-{}'.format(name, uname_dict[name])
    if ' home' in name:
        name.replace(' home', '-home')
                
    info = {}
    
    medulla.add_Neuron(name, # uname
                         cell_type, # name
                         referenceId = str(bodyID), #referenceId
                         info = info if len(info) else None,
                         morphology = {'type': 'swc', 'filename': '{}/{}.swc'.format(swc_dir, bodyID), 'scale': 0.001*10},)

In [ ]:
neurons = medulla.sql_query('select from Neuron').nodes_as_objs
# set the cache so there is no need for database access.
for neuron in neurons:
    medulla.set('Neuron', neuron.uname, neuron, medulla.default_DataSource)
neuron_ref_to_obj = {int(neuron.referenceId): neuron for neuron in neurons}

In [ ]:
synapse_df = pd.read_csv('synapses.csv')

In [ ]:
for i, row in tqdm(synapse_df.iterrows()):
    pre_neuron = neuron_ref_to_obj[row['pre_id']]
    post_neuron = neuron_ref_to_obj[row['post_id']]

    pre_conf = np.array(eval(row['pre_confidence']))/1e6
    post_conf = np.array(eval(row['post_confidence']))/1e6
    NHP = np.sum(np.logical_and(post_conf>=0.7, pre_conf>=0.7))

    content = {'type': 'swc'}
    content['x'] = [round(i, 3) for i in (np.array(eval(row['pre_x'])+eval(row['post_x']))/1000.*10).tolist()]
    content['y'] = [round(i, 3) for i in (np.array(eval(row['pre_y'])+eval(row['post_y']))/1000.*10).tolist()]
    content['z'] = [round(i, 3) for i in (np.array(eval(row['pre_z'])+eval(row['post_z']))/1000.*10).tolist()]
    content['r'] = [0]*len(content['x'])
    content['parent'] = [-1]*(len(content['x'])//2) + [i+1 for i in range(len(content['x'])//2)]
    content['identifier'] = [7]*(len(content['x'])//2) + [8]*(len(content['x'])//2)
    content['sample'] = [i+1 for i in range(len(content['x']))]
    content['confidence'] = [round(i, 3) for i in pre_conf.tolist()] + [round(i, 3) for i in post_conf.tolist()]
    
    medulla.add_Synapse(pre_neuron, post_neuron, N = row['N'], NHP = NHP,
                          morphology = content)
                          #arborization = arborization)
        


## Figure out Arborization Data from Loaded Synapse Positions

In [ ]:
def strata_arborization(z):
    sep = np.array([20, 28, 35, 38, 42, 44, 55, 58, 70])
    return Counter(np.digitize(z, sep))

In [ ]:

neuron_dict = {}

for neuron in neurons:
    neuron_dict[neuron.uname] = {'axons': Counter(), 'dendrites': Counter(), 'obj': neuron} 
    
for neuron in tqdm(neurons):
    outgoing_synapses = medulla.sql_query("""select expand(out('SendsTo')) from {}""".format(neuron._id)).nodes_as_objs
    for synapse in outgoing_synapses:
        morphology = [n for n in synapse.out('HasData') if isinstance(n, models.MorphologyData)][0]
        arborization = []
        arborization.append({'type': 'neuropil',
                             'synapses': {'MED(L)': len(morphology.x)}})
        s = strata_arborization(morphology.z[:(len(morphology.z)//2)])
        arborization.append({'type': 'subregion',
                             'synapses': {'MED-M{}(L)'.format(k+1): v for k, v in s.items()}})
        neuron_dict[neuron.uname]['axons'] += s
        neuron_dict[synapse.out('SendsTo')[0].uname]['dendrites'] += s
        medulla.add_synapse_arborization(synapse, arborization)


In [ ]:
for neuron in tqdm(neurons):
    arborization = []
    arborization.append({'type': 'neuropil',
                         'dendrites': {'MED(L)': sum(neuron_dict[neuron.uname]['dendrites'].values())},
                         'axons': {'MED(L)': sum(neuron_dict[neuron.uname]['axons'].values())}})
    arborization.append({'type': 'subregion',
                         'dendrites': {'MED-M{}(L)'.format(k+1): v for k, v in neuron_dict[neuron.uname]['dendrites'].items()},
                         'axons': {'MED-M{}(L)'.format(k+1): v for k, v in neuron_dict[neuron.uname]['axons'].items()}})
    medulla.add_neuron_arborization(neuron, arborization)
    
    